In [7]:
import os
import numpy as np
import json
import sys
sys.path.append("../utils")
from reader import read_record
from spectrogram import save_spectrogram  
from tqdm import tqdm

In [8]:
RAW_DIR = "../data/raw"
PROCESSED_DIR = "../data/processed"
SPEC_DIR = "../data/spectrograms"
CHANNEL = "EEG Fpz-Cz"

In [9]:
def find_matching_hypnogram(psg_file, raw_dir):
    base = os.path.basename(psg_file).split("-")[0][:7]  # e.g., SC4001E
    for fname in os.listdir(raw_dir):
        if fname.startswith(base) and fname.endswith("-Hypnogram.edf"):
            return os.path.join(raw_dir, fname)
    return None

In [10]:
def process_subject(subj_id):
    try:
        psg_file = os.path.join(RAW_DIR, f"{subj_id}-PSG.edf")
        if not os.path.exists(psg_file):
            print(f"{subj_id} ❌ PSG file not found")
            return

        hyp_file = find_matching_hypnogram(psg_file, RAW_DIR)
        if not hyp_file or not os.path.exists(hyp_file):
            print(f"{subj_id} ❌ Hypnogram file not found")
            return

        spec_path = os.path.join(SPEC_DIR, subj_id)
        os.makedirs(spec_path, exist_ok=True)

        raw, events = read_record(psg_file, hyp_file, channel=CHANNEL)
        signal = raw.get_data()[0]
        sfreq = raw.info['sfreq']
        samples_per_epoch = int(30 * sfreq)

        X = np.array([
            signal[i * samples_per_epoch:(i + 1) * samples_per_epoch]
            for i in range(len(events))
        ])
        y = np.array([e[2] for e in events])

        np.savez(os.path.join(PROCESSED_DIR, f"{subj_id}_data.npz"), X=X, y=y)

        for i in range(len(X)):
            save_path = os.path.join(spec_path, f"epoch_{i:04d}.png")
            save_spectrogram(X[i], save_path)

        label_dict = {f"epoch_{i:04d}.png": int(y[i]) for i in range(len(y))}
        with open(os.path.join(spec_path, "labels.json"), "w") as f:
            json.dump(label_dict, f)

        print(f"{subj_id} ✅ done")

    except Exception as e:
        print(f"{subj_id} ❌ failed: {e}")

In [11]:
all_subjs = sorted(set([
    f.split("-")[0]
    for f in os.listdir(RAW_DIR)
    if "-PSG.edf" in f
]))

In [12]:
completed = []
failed = []

for subj_id in tqdm(all_subjs):
    try:
        process_subject(subj_id)
        completed.append(subj_id)
    except Exception as e:
        print(f"{subj_id} ❌ failed: {e}")
        failed.append(subj_id)

print("✅ Done subjects:", completed)
print("❌ Failed subjects:", failed)

  0%|          | 0/153 [00:00<?, ?it/s]

Extracting EDF parameters from d:\UNI\پروژه پایانی\Sleep Stage Project 1.0\data\raw\SC4001E0-PSG.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 7949999  =      0.000 ... 79499.990 secs...


d:\UNI\پروژه پایانی\Sleep Stage Project 1.0\notebooks\../utils\reader.py:19: RuntimeWarning: Channels contain different highpass filters. Highest filter setting will be stored.
  raw = mne.io.read_raw_edf(psg_path, preload=True)
d:\UNI\پروژه پایانی\Sleep Stage Project 1.0\notebooks\../utils\reader.py:19: RuntimeWarning: Channels contain different lowpass filters. Lowest filter setting will be stored.
  raw = mne.io.read_raw_edf(psg_path, preload=True)
d:\UNI\پروژه پایانی\Sleep Stage Project 1.0\notebooks\../utils\reader.py:19: RuntimeWarning: Highpass cutoff frequency 16.0 is greater than lowpass cutoff frequency 0.7, setting values to 0 and Nyquist.
  raw = mne.io.read_raw_edf(psg_path, preload=True)


NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


d:\UNI\پروژه پایانی\Sleep Stage Project 1.0\notebooks\../utils\reader.py:21: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw.set_annotations(annotations)


...

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


d:\UNI\پروژه پایانی\Sleep Stage Project 1.0\notebooks\../utils\reader.py:21: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw.set_annotations(annotations)


Used Annotations descriptions: [np.str_('Sleep stage 1'), np.str_('Sleep stage 2'), np.str_('Sleep stage 3'), np.str_('Sleep stage 4'), np.str_('Sleep stage ?'), np.str_('Sleep stage R'), np.str_('Sleep stage W')]


100%|██████████| 153/153 [46:37<00:00, 18.28s/it]

SC4822G0 ✅ done
✅ Done subjects: ['SC4001E0', 'SC4002E0', 'SC4011E0', 'SC4012E0', 'SC4021E0', 'SC4022E0', 'SC4031E0', 'SC4032E0', 'SC4041E0', 'SC4042E0', 'SC4051E0', 'SC4052E0', 'SC4061E0', 'SC4062E0', 'SC4071E0', 'SC4072E0', 'SC4081E0', 'SC4082E0', 'SC4091E0', 'SC4092E0', 'SC4101E0', 'SC4102E0', 'SC4111E0', 'SC4112E0', 'SC4121E0', 'SC4122E0', 'SC4131E0', 'SC4141E0', 'SC4142E0', 'SC4151E0', 'SC4152E0', 'SC4161E0', 'SC4162E0', 'SC4171E0', 'SC4172E0', 'SC4181E0', 'SC4182E0', 'SC4191E0', 'SC4192E0', 'SC4201E0', 'SC4202E0', 'SC4211E0', 'SC4212E0', 'SC4221E0', 'SC4222E0', 'SC4231E0', 'SC4232E0', 'SC4241E0', 'SC4242E0', 'SC4251E0', 'SC4252E0', 'SC4261F0', 'SC4262F0', 'SC4271F0', 'SC4272F0', 'SC4281G0', 'SC4282G0', 'SC4291G0', 'SC4292G0', 'SC4301E0', 'SC4302E0', 'SC4311E0', 'SC4312E0', 'SC4321E0', 'SC4322E0', 'SC4331F0', 'SC4332F0', 'SC4341F0', 'SC4342F0', 'SC4351F0', 'SC4352F0', 'SC4362F0', 'SC4371F0', 'SC4372F0', 'SC4381F0', 'SC4382F0', 'SC4401E0', 'SC4402E0', 'SC4411E0', 'SC4412E0', 'SC442